In [1]:
%load_ext autoreload
%autoreload 2
import wallaroo

import os, json
os.environ['DEPLOYMENT_CONFIG'] = json.dumps({"cpus": 0.5, "replica_count": 1, "memory": "500Mi"})

auth_type = "sso"
client = wallaroo.Client(auth_type=auth_type, request_timeout=90, interactive=True)

In [2]:
# for p in client.list_pipelines(): p.undeploy()

Waiting for undeployment - this will take up to 90s ..................................... ok
 ok
 ok


In [3]:
import pyarrow as pa
input_schema = pa.schema([
    pa.field('dense_2', pa.list_(pa.float64()))
])
output_schema = pa.schema([
    pa.field('output', pa.list_(pa.float64()))
])

In [4]:
step = client.upload_model("python-step", "./step.py").configure('python', input_schema=input_schema, output_schema=output_schema)


In [5]:
# Deploy a model/pipeline

pipeline_name = 'py-shape3'
model_name = 'py-shape-onnx'

onnx_file_model_name = "house_price_keras.onnx"

fraud = client.upload_model(model_name, onnx_file_model_name).configure('onnx')
pipeline = client.build_pipeline(pipeline_name)
pipeline = pipeline.add_model_step(fraud)

pipeline = pipeline.add_model_step(step)
deployment = pipeline.deploy()

Waiting for deployment - this will take up to 90s ............ ok


In [7]:
import pandas as pd

data = pd.DataFrame.from_dict({"tensor": [[0.6878518042239091,
 0.17607340208535074,
 -0.8695140830357148,
 0.34638762962802144,
 -0.0916270832672289,
 -0.022063226781124278,
 -0.13969884765926363,
 1.002792335666138,
 -0.3067449033633758,
 0.9272000630461978,
 0.28326687982544635,
 0.35935375728372815,
 -0.682562654045523,
 0.532642794275658,
 -0.22705189652659302,
 0.5743846356405602,
 -0.18805086358065454]]})

,time,in.tensor,out.output,check_failures
0,2023-06-15 13:55:50.300,"[0.6878518042, 0.1760734021, -0.869514083, 0.3...",[12.886651039123535],0


In [ ]:
deployment.infer(data)

In [8]:
deployment.logs()

,time,in.tensor,out.output,check_failures
0,2023-06-15 13:55:50.300,"[0.6878518042, 0.1760734021, -0.869514083, 0.3...",[12.886651039123535],0
